In [1]:
# 기업명으로 10개분기 재무제표 다운로드: page_count = 10(디폴트값) 임

import requests
import pandas as pd
from io import BytesIO
import re
from zipfile import ZipFile
import xmltodict
import json

def dload_dart_10q(corp_name):
    api_key = '7d523a1a3ce98201eb9eeb5c04c206363c176810'
    url_corp = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={}'.format(api_key)
    resp_corp = requests.get(url_corp)

    zf_corp = ZipFile(BytesIO(resp_corp.content))
    zf_corp_name = zf_corp.namelist()
    zf_corp_name_pop = zf_corp_name.pop()

    data_xml = zf_corp.read(zf_corp_name_pop).decode('utf-8')
    data_odict = xmltodict.parse(data_xml)
    data_dict = json.loads(json.dumps(data_odict))
    data = data_dict.get('result', {}).get('list')

    where_name = (item for item in data if item['corp_name'] == corp_name)
    # corp_name 이 corp_name인 dict를 list에서 가져오기
    # 만약 값이 없을 경우 false가 dict에 저장됨
    dict_name = next(where_name, False)
    corp_code = dict_name['corp_code']

    print(corp_code)
    
    url_info = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de=19990101&last_reprt_at=N&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003'.format(api_key, corp_code)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
    headers = {'user-agent' : user_agent}

    resp = requests.get(url_info)
    resp_decode = resp.content.decode('utf-8')

    period = re.findall(r'<report_nm>(.*?)</report_nm>', resp_decode)
    rcp = re.findall(r'<rcept_no>(.*?)</rcept_no>', resp_decode)
    # <report_nm> 이런건 미리 해보고 알 수 있는 것들
    # company는 아직... 좀 더 해보고...

    dcm = []

    for i in range(len(rcp)):
        url_rcp = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rcp[i]
        resp_rcp = requests.get(url_rcp, headers = headers)
        resp_rcp_decode = resp_rcp.content.decode('utf-8')
        dcm_i = re.findall(rcp[i]+"', '(.*?)',", resp_rcp_decode)[0]
        dcm = dcm + [dcm_i]
        
    dict2_info = dict(zip(zip(period, rcp),dcm))
    
    print(period[0])
    print(rcp[0])
    print(dcm[0])
    
    for i in range(min(len(period), len(rcp), len(dcm))):
        url_rcpdcm = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no='+rcp[i]+'&dcm_no='+dcm[i]+'&lang=ko'
        sheets = ['연결 재무상태표', '연결 손익계산서', '연결 포괄손익계산서']
        # sheets의 경우 지금은 이렇게 하지만 나중엔 어떤 sheet가 있는지도 알아낼 수 있어야 할 것 같음
               
        for sheet in sheets:
            resp_rcpdcm = requests.get(url_rcpdcm, headers = headers)
            output = BytesIO(resp_rcpdcm.content)
            table = pd.read_excel(output, sheet_name = sheet, skiprows = 5)
            table.to_csv(period[i]+corp_name+sheet+".csv", encoding = 'euc-kr')


In [2]:
dload_dart_10q('현대자동차')

00164742
분기보고서 (2020.09)
20201116001588
7549889
